## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

<hr style="border:2px solid black"> </hr>

## Model class and functions

Function to get random initial cluster centers

In [ ]:
def get_random_cluster_centers(X, n_clusters):
    cluster_centers_indices = []
    c = 0
    while c < n_clusters:
        new_center_index = np.random.randint(0, X.shape[0])
        if new_center_index not in cluster_centers_indices:
            cluster_centers_indices.append(new_center_index)
            c += 1
    cluster_centers = []
    for i in range(len(cluster_centers_indices)):
        cluster_centers.append(X[cluster_centers_indices[i]])
    return np.array(cluster_centers)

Function to get the distances between 2 points

In [ ]:
def get_distance(p1, p2):
    d = p1 - p2
    d = np.square(d)
    d = np.sum(d)
    d = np.sqrt(d)
    return d

Function to get the distances between points and a cluster center

In [ ]:
def get_distances_to_cluster_center(X, cluster_center):
    d = X - cluster_center
    d = np.square(d)
    d = np.sum(d, axis=1)
    d = np.sqrt(d)
    return d

Function to get the distances between points and all cluster centers

In [ ]:
def get_distances_to_cluster_centers(X, cluster_centers):
    distances_to_centers = []
    for i in range(cluster_centers.shape[0]):
        distances_to_center = get_distances_to_cluster_center(X, cluster_centers[i])
        distances_to_centers.append(distances_to_center)
    return np.array(distances_to_centers)

Function to get the label of the cluster that each point belongs to

In [ ]:
def get_cluster_labels(distances_to_centers):
    return np.argmin(distances_to_centers, axis=0)

In [ ]:
def get_x_clusters(X, labels, n_clusters):
    X_clusters = []
    for i in range(n_clusters):
        X_clusters.append([])
    for i in range(X.shape[0]):
        X_clusters[labels[i]].append(X[i])
    for i in range(n_clusters):
        X_clusters[i] = np.array(X_clusters[i])
    return X_clusters

Function to get the centers of the clusters based on the labels

In [ ]:
def get_clusters_centers(X, labels, n_clusters):
    X_clusters = get_x_clusters(X, labels, n_clusters)
    cluster_centers = []
    for i in range(n_clusters):
        cluster_center = np.mean(X_clusters[i], axis=0)
        cluster_centers.append(cluster_center)
    return np.array(cluster_centers)

Function to calculate the DBI of the clustering result

In [ ]:
def calculate_DBI(X, labels, cluster_centers):
    n_clusters = cluster_centers.shape[0]
    X_clusters = get_x_clusters(X, labels, n_clusters)

    s = []
    for i in range(n_clusters):
        distances_to_center = get_distances_to_cluster_center(X_clusters[i], cluster_centers[i])
        s_i = np.sum(distances_to_center) / (X_clusters[i].shape[0])
        s.append(s_i)
    
    r = []
    for i in range(n_clusters):
        r_i = []
        for j in range(n_clusters):
            if i < j:
                d_ij = get_distance(cluster_centers[i], cluster_centers[j])
                r_ij = (s[i] + s[j]) / d_ij
            elif i > j:
                r_ij = r[j][i]
            else:
                r_ij = -np.inf
            r_i.append(r_ij)
        r.append(r_i)
    
    dbi = 0
    for i in range(n_clusters):
        dbi += np.amax(r[i])
    dbi /= n_clusters
    
    return dbi

Function to get the confusion matrix

In [ ]:
def get_confusion_matrix(Y_test, Y_predict, labels):
    number_of_classes = len(labels)
    confusion_matrix = np.zeros((number_of_classes, number_of_classes))
    for i in range(len(Y_test)):
        row = labels.index(Y_test[i])
        column = labels.index(Y_predict[i])
        confusion_matrix[row][column] += 1
    return confusion_matrix

K-means model class

In [ ]:
class KMeans:
    def __init__(self):
        self.labels = []
        self.cluster_centers = []
    
    def fit(self, X, n_clusters): 
        cluster_centers = get_random_cluster_centers(X, n_clusters)
        while True:
            distances_to_centers = get_distances_to_cluster_centers(X, cluster_centers)
            labels = get_cluster_labels(distances_to_centers)
            new_cluster_centers = get_clusters_centers(X, labels, n_clusters)
            if np.array_equal(new_cluster_centers, cluster_centers):
                break
            cluster_centers = new_cluster_centers
        self.labels = labels
        self.cluster_centers = cluster_centers

K-means classifier class:

In [ ]:
class KMeansClassifier:
    def __init__(self):
        self.labels = []
        self.cluster_centers = []
        self.max_counts = []
    
    def fit(self, X, Y, labels, n_clusters):
        model = KMeans()
        model.fit(X, n_clusters)
        self.fit_centers(X, Y, labels, model.labels, model.cluster_centers)

    def fit_centers(self, X, Y, labels, cluster_labels, cluster_centers):
        X_classes = []
        X_classes_cluster_labels = []
        for i in range(len(labels)):
            X_classes.append([])
            X_classes_cluster_labels.append([])
        for i in range(X.shape[0]):
            for j in range(len(labels)):
                if Y[i] == labels[j]:
                    X_classes[j].append(X[i])
                    X_classes_cluster_labels[j].append(cluster_labels[i])
                    break

        classes_cluster_centers = []
        max_counts = []
        for i in range(len(labels)):
            class_counts = np.zeros(len(labels), dtype=np.uint16)
            for j in range(len(X_classes[i])):
                cluster_label = X_classes_cluster_labels[i][j]
                class_counts[cluster_label] += 1
            class_cluster = np.argmax(class_counts)
            classes_cluster_centers.append(cluster_centers[class_cluster])
            max_counts.append(class_counts[class_cluster])
        
        self.labels = labels
        self.cluster_centers= np.array(classes_cluster_centers)
        self.max_counts= np.array(max_counts)

    def predict(self, X):
        distances_to_centers = get_distances_to_cluster_centers(X, self.cluster_centers)
        target_values = np.argmin(distances_to_centers, axis=0)
        Y_predict = []
        for i in range(len(target_values)):
            Y_predict.append(self.labels[target_values[i]])
        return Y_predict

<hr style="border:2px solid black"> </hr>

## Getting the images and their labels

Read training and testing images

In [ ]:
airplane_train_images = np.array([plt.imread("Data/Train/airplane/"+str(i)+".jpg") for i in range(0, 5000)], dtype=np.int64)
bird_train_images = np.array([plt.imread("Data/Train/bird/"+str(i)+".jpg") for i in range(0, 5000)], dtype=np.int64)
truck_train_images = np.array([plt.imread("Data/Train/truck/"+str(i)+".jpg") for i in range(0, 5000)], dtype=np.int64)

airplane_test_images = np.array([plt.imread("Data/Test/airplane/"+str(i)+".jpg") for i in range(0, 1000)], dtype=np.int64)
bird_test_images = np.array([plt.imread("Data/Test/bird/"+str(i)+".jpg") for i in range(0, 1000)], dtype=np.int64)
truck_test_images = np.array([plt.imread("Data/Test/truck/"+str(i)+".jpg") for i in range(0, 1000)], dtype=np.int64)

Transforming the images into feature vectors

In [ ]:
X_airplane_train = np.array([airplane_train_images[i].reshape(-1) for i in range(0, airplane_train_images.shape[0])])
X_bird_train = np.array([bird_train_images[i].reshape(-1) for i in range(0, bird_train_images.shape[0])])
X_truck_train = np.array([truck_train_images[i].reshape(-1) for i in range(0, truck_train_images.shape[0])])

X_airplane_test = np.array([airplane_test_images[i].reshape(-1) for i in range(0, airplane_test_images.shape[0])])
X_bird_test = np.array([bird_test_images[i].reshape(-1) for i in range(0, bird_test_images.shape[0])])
X_truck_test = np.array([truck_test_images[i].reshape(-1) for i in range(0, truck_test_images.shape[0])])

X_train = np.concatenate((X_airplane_train, X_bird_train, X_truck_train))
X_test = np.concatenate((X_airplane_test, X_bird_test, X_truck_test))

Getting labels for all the classes train and test data

In [ ]:
Y_train = ["airplane" for i in range(airplane_train_images.shape[0])]
Y_train += ["bird" for i in range(bird_train_images.shape[0])]
Y_train += ["truck" for i in range(truck_train_images.shape[0])]

Y_test = ["airplane" for i in range(airplane_test_images.shape[0])]
Y_test += ["bird" for i in range(bird_test_images.shape[0])]
Y_test += ["truck" for i in range(truck_test_images.shape[0])]

labels = ["airplane", "bird", "truck"]

<hr style="border:2px solid black"> </hr>

## Training and testing the models

Running K-means multiple times with random initilizations

In [ ]:
models = []
dbi_values = []
for i in range(10):
    model = KMeans()
    model.fit(X_train, n_clusters=3)
    models.append(model)
    dbi = calculate_DBI(X_train, model.labels, model.cluster_centers)
    dbi_values.append(dbi)

Getting the model with the lowest DBI value

In [ ]:
best_model = models[np.argmin(dbi_values)]

Create and test a K-means classifier

In [ ]:
classifier = KMeansClassifier()
classifier.fit_centers(X_train, Y_train, labels, best_model.labels, best_model.cluster_centers)
Y_predict = classifier.predict(X_test)

<hr style="border:2px solid black"> </hr>

## Plotting the results

In [ ]:
max_counts = classifier.max_counts
confusion_matrix = get_confusion_matrix(Y_test, Y_predict, labels)

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(['Airplane','Bird','Truck'],max_counts,'-o')
plt.title('Best Counts')
plt.savefig("figure_best_counts")

In [ ]:
plt.rc('figure', figsize=[5,5])
plt.matshow(confusion_matrix,cmap="Blues")
for i in range(0,confusion_matrix.shape[0]):
    for j in range(0,confusion_matrix.shape[1]):
        plt.annotate(confusion_matrix[i,j],(j,i))
plt.savefig("figure_confusion_matrix")